In [27]:
%matplotlib inline


Word Embeddings: Encoding Lexical Semantics
===========================================

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


Word Embeddings in Pytorch
~~~~~~~~~~~~~~~~~~~~~~~~~~

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [28]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#设置确定的seed，每次运行结果都是相同可对比验证的
torch.manual_seed(1)

In [29]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


An Example: N-Gram Language Modeling
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




In [30]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        print(inputs)
        embeds = self.embeddings(inputs).view((1, -1))
        print(embeds)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1) #dim=1代表按行计算softmax
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
tensor([13, 40])
tensor([[ 0.5514, -1.5474,  0.7575, -0.4068, -0.1277,  0.2804,  1.7460,  1.8550,
         -0.7064,  2.5571, -0.3761, -2.4107, -1.2778, -0.0629, -1.2308, -1.0655,
         -0.3865,  0.3782, -0.2076,  0.6936]], grad_fn=<ViewBackward>)
tensor([40, 28])
tensor([[-0.3761, -2.4107, -1.2778, -0.0628, -1.2308, -1.0656, -0.3864,  0.3782,
         -0.2076,  0.6936,  0.7879,  1.3686, -0.8507,  0.5126,  0.3331,  1.1079,
          0.1857, -0.2764, -0.6116,  0.8160]], grad_fn=<ViewBackward>)
tensor([28,  0])
tensor([[ 0.7878,  1.3685, -0.8508,  0.5127,  0.3331,  1.1079,  0.1857, -0.2765,
         -0.6116,  0.8159,  0.8657,  0.2444, -0.6629,  0.8073,  0.4391,  1.1712,
          1.7674, -0.0954,  0.0612, -0.6177]], grad_fn=<ViewBackward>)
tensor([ 0, 57])
tensor([[ 0.8657,  0.2444, -0.6629,  0.8073,  0.4392,  1.1711,  1.7674, -0.0953,
          0.0612, -0.6177,  0.2204, -1.9548, -1.405

tensor([78,  8])
tensor([[-0.8254, -0.4085, -2.3702, -0.0215, -0.3100, -1.0479, -0.9336,  0.2453,
          0.1793,  0.7637,  1.3852, -0.8140, -0.9276,  1.1119,  0.6156,  0.1938,
         -2.5831,  0.8540, -2.1021, -0.6200]], grad_fn=<ViewBackward>)
tensor([ 8, 50])
tensor([[ 1.3852, -0.8140, -0.9276,  1.1119,  0.6155,  0.1938, -2.5831,  0.8539,
         -2.1021, -0.6199, -0.0368,  0.1819, -1.2673, -0.5944,  0.4527,  0.3156,
         -0.6901, -0.2829,  0.0990,  0.4938]], grad_fn=<ViewBackward>)
tensor([50, 79])
tensor([[-0.0369,  0.1819, -1.2674, -0.5944,  0.4526,  0.3156, -0.6901, -0.2828,
          0.0988,  0.4938, -0.1925, -0.6860, -0.7390,  0.2279,  0.9885, -2.1412,
         -0.2242, -0.0657, -1.2894,  0.0591]], grad_fn=<ViewBackward>)
tensor([79, 30])
tensor([[-0.1925, -0.6860, -0.7389,  0.2280,  0.9886, -2.1412, -0.2242, -0.0657,
         -1.2894,  0.0591,  0.1437,  0.2032,  1.0541,  0.1317,  1.4023,  0.4839,
         -0.7030, -0.8268,  0.1012,  0.1952]], grad_fn=<ViewBackward>)


tensor([15, 96])
tensor([[ 0.5231,  1.2238, -0.4032, -0.9590, -0.0054, -0.0787, -0.3894, -0.0798,
          0.7607, -1.0024, -1.2944, -0.1284,  1.0658, -2.3534,  1.1661,  0.1702,
          0.5287,  2.8890,  0.1600, -0.3637]], grad_fn=<ViewBackward>)
tensor([96, 27])
tensor([[-1.2944, -0.1283,  1.0659, -2.3534,  1.1661,  0.1701,  0.5286,  2.8889,
          0.1600, -0.3637, -0.2690,  1.3117, -1.5565, -1.0755, -0.8751, -0.4727,
         -0.7530, -0.4319,  0.6693,  0.6505]], grad_fn=<ViewBackward>)
tensor([27, 86])
tensor([[-0.2689,  1.3116, -1.5566, -1.0755, -0.8751, -0.4727, -0.7530, -0.4319,
          0.6693,  0.6505,  1.1450,  1.0682, -0.9495, -0.7223,  0.2305, -0.1481,
         -0.6361,  0.3536, -0.0964,  1.3375]], grad_fn=<ViewBackward>)
tensor([86, 37])
tensor([[ 1.1450,  1.0683, -0.9496, -0.7223,  0.2305, -0.1482, -0.6362,  0.3537,
         -0.0964,  1.3374,  1.2561,  0.4998,  0.0400,  0.4647, -3.3792,  0.9734,
         -0.6191, -0.6363, -0.4242, -2.0270]], grad_fn=<ViewBackward>)


tensor([39, 38])
tensor([[ 0.3454, -0.5508, -0.9605,  0.2767, -0.6801, -0.5452,  1.1529, -0.3996,
          0.9738,  1.5452,  1.3018, -0.6293, -0.0681,  0.6942, -0.2891,  0.1519,
         -0.2975, -0.1309,  1.5366,  0.9197]], grad_fn=<ViewBackward>)
tensor([38, 12])
tensor([[ 1.3018, -0.6293, -0.0681,  0.6941, -0.2892,  0.1518, -0.2974, -0.1309,
          1.5366,  0.9197,  2.2953,  0.6750,  1.7132, -1.7946, -1.5212,  0.9194,
         -0.5485, -0.3468,  0.4729, -0.4288]], grad_fn=<ViewBackward>)
tensor([12,  7])
tensor([[ 2.2953,  0.6750,  1.7133, -1.7946, -1.5213,  0.9194, -0.5486, -0.3467,
          0.4728, -0.4289, -0.3418, -0.3007, -1.0485, -0.4707,  0.2913,  1.9910,
         -0.9244, -0.9300,  1.4301,  0.4210]], grad_fn=<ViewBackward>)
tensor([ 7, 60])
tensor([[-0.3418, -0.3008, -1.0485, -0.4707,  0.2913,  1.9910, -0.9243, -0.9299,
          1.4301,  0.4211, -0.4276,  1.2989, -1.0202, -0.8528, -1.3629, -0.1732,
          1.5375, -1.1706, -1.0076, -1.1257]], grad_fn=<ViewBackward>)


tensor([26, 82])
tensor([[-0.8334, -1.1923, -2.3057,  0.6034,  0.3149,  1.1419,  0.3050, -0.5789,
          0.5647, -0.8770, -1.2692,  0.3293,  0.3711, -1.0254, -2.2673,  0.0935,
          0.4827,  0.0830,  0.9542, -0.6846]], grad_fn=<ViewBackward>)
tensor([82, 34])
tensor([[-1.2693,  0.3293,  0.3712, -1.0254, -2.2673,  0.0935,  0.4827,  0.0830,
          0.9543, -0.6846, -0.4327, -0.1147,  1.7961, -0.4776, -2.7180, -0.2221,
          0.3419,  1.1089, -0.5273,  0.0850]], grad_fn=<ViewBackward>)
tensor([34, 85])
tensor([[-0.4327, -0.1147,  1.7961, -0.4777, -2.7179, -0.2221,  0.3418,  1.1089,
         -0.5273,  0.0851,  0.0812,  1.0828, -0.7216,  0.6994, -0.6602,  0.5687,
          0.8858, -0.8949,  0.1355,  0.6577]], grad_fn=<ViewBackward>)
tensor([85, 42])
tensor([[ 0.0811,  1.0829, -0.7216,  0.6994, -0.6601,  0.5687,  0.8857, -0.8948,
          0.1355,  0.6577, -0.0078,  2.6161, -0.0640,  0.2609, -0.3604, -1.2615,
          1.9152, -1.8606,  0.4742,  0.9986]], grad_fn=<ViewBackward>)


tensor([44,  2])
tensor([[-3.2689,  0.4743, -2.1141, -1.5000,  0.2693, -0.4885,  1.3678,  0.4537,
          1.1750, -0.3415, -1.8823, -0.7768,  2.0239, -0.0859,  2.3575, -1.0382,
          1.5744, -0.6293,  2.4072,  0.2782]], grad_fn=<ViewBackward>)
tensor([ 2, 58])
tensor([[-1.8823, -0.7768,  2.0239, -0.0859,  2.3576, -1.0383,  1.5744, -0.6292,
          2.4073,  0.2781,  0.4879, -0.3089, -3.0145, -1.2491,  1.3488,  0.2681,
         -1.1276, -0.6002,  1.8372, -1.0710]], grad_fn=<ViewBackward>)
tensor([58, 92])
tensor([[ 0.4878, -0.3090, -3.0146, -1.2491,  1.3487,  0.2681, -1.1275, -0.6003,
          1.8371, -1.0709,  0.9840, -1.8280, -0.6589, -1.5521, -0.0038,  0.1425,
          0.9432, -0.2260,  0.9326,  0.8788]], grad_fn=<ViewBackward>)
tensor([92, 68])
tensor([[ 0.9840, -1.8281, -0.6588, -1.5522, -0.0038,  0.1426,  0.9432, -0.2261,
          0.9326,  0.8789,  0.4179,  0.9272,  0.6353, -0.1601,  0.1684,  0.7757,
         -1.8132, -0.3154, -1.5043, -0.1864]], grad_fn=<ViewBackward>)


tensor([[ 0.2507,  0.3680, -0.5096, -0.1001,  0.3793,  0.4167, -0.7539,  0.7917,
          0.0766, -1.0782, -0.3456,  1.6416, -0.0583,  0.3322, -0.2779,  0.0997,
         -1.5760,  0.7489,  1.9228,  1.4200]], grad_fn=<ViewBackward>)
tensor([66, 83])
tensor([[-0.3456,  1.6416, -0.0582,  0.3323, -0.2779,  0.0997, -1.5761,  0.7490,
          1.9227,  1.4200,  0.0349,  0.5593,  0.1959, -0.9501,  0.1306,  0.8639,
         -0.3116,  0.0242, -0.3932,  0.3102]], grad_fn=<ViewBackward>)
tensor([83, 77])
tensor([[ 0.0349,  0.5593,  0.1960, -0.9501,  0.1306,  0.8640, -0.3117,  0.0241,
         -0.3933,  0.3102, -0.2164,  0.3324, -0.9342,  1.6585,  1.0067,  1.3929,
         -1.7941,  1.3676,  0.0222,  0.9847]], grad_fn=<ViewBackward>)
tensor([77, 29])
tensor([[-0.2165,  0.3324, -0.9343,  1.6585,  1.0066,  1.3930, -1.7941,  1.3676,
          0.0223,  0.9847,  0.2476, -0.3854, -0.1999, -0.8377, -1.5528, -0.0576,
          1.1984, -2.2823, -1.0137, -0.8883]], grad_fn=<ViewBackward>)
tensor([29, 48])


tensor([[-0.3882,  0.5481, -0.8019,  0.3028, -2.0321, -1.1075,  0.1572, -0.5977,
         -0.8830,  0.6076,  0.1635, -0.3128,  0.1252, -0.5269,  0.7167,  0.3965,
          0.1949,  0.3424, -0.4342,  0.2773]], grad_fn=<ViewBackward>)
tensor([48, 73])
tensor([[ 0.1636, -0.3127,  0.1253, -0.5270,  0.7167,  0.3965,  0.1949,  0.3424,
         -0.4342,  0.2774, -1.3571, -0.9610, -0.3545,  1.3510,  1.8442, -1.2513,
         -1.4713, -1.4516,  0.2560, -0.7526]], grad_fn=<ViewBackward>)
tensor([73, 65])
tensor([[-1.3571, -0.9610, -0.3545,  1.3509,  1.8442, -1.2513, -1.4713, -1.4515,
          0.2561, -0.7525, -0.9745,  0.0994,  0.4418, -0.5249,  0.4936, -0.5375,
         -0.6989,  0.5481, -1.0923, -0.1997]], grad_fn=<ViewBackward>)
tensor([65, 49])
tensor([[-0.9745,  0.0993,  0.4417, -0.5249,  0.4936, -0.5376, -0.6990,  0.5482,
         -1.0924, -0.1997,  0.2942, -0.5639,  0.7258, -0.5460, -0.8028,  0.0186,
         -0.1515,  0.5248, -0.2125, -1.2543]], grad_fn=<ViewBackward>)
tensor([49, 45])


Exercise: Computing Word Embeddings: Continuous Bag-of-Words
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




In [31]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 10
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

model=CBOW()

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
